In [294]:
from bs4 import BeautifulSoup
import requests
import re
import csv

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [295]:
class FaroesProperties:
    def __init__(self, website=None, address=None, price=None,
                 LatesPrice=None, validDate=None,
                 date=None, buildingSize=None,
                 landSize=None, room=None,
                 floor=None):
        if website is None:
            self.websites = "None"
        self.websites = website
        if address is None:
            self.addresses = "None"
        self.addresses = address
        if prices is None:
            self.prices = "None"
        self.prices = price
        if LatestPrices is None:
            self.LatestPrices = "None"
        self.LatestPrices = LatesPrice
        if validDates is None:
            self.validDates = "None"
        self.validDates = validDate
        if dates is None:
            self.dates = "None"
        self.dates = date
        if buildingSizes is None:
            self.buildingSizes = "None"
        self.buildingSizes = buildingSize
        if landSize is None:
            self.landSizes = "None"
        self.landSizes = landSize
        if rooms is None:
            self.rooms = "None"
        self.rooms = room
        if floors is None:
            self.floors = "None"
        self.floors = floor
    def display(self):
        print("Heimasída: ", self.websites, "\nAddressa: ", self.addresses,"\nPrísur: ", self.prices,"\nSeinasti bod: ",
              self.LatestPrices, "\nGaldandi til dato: ", self.validDates,"\nDato: ", self.dates,
              "\nm2 í húsinum: ", self.buildingSizes,"\nm2 á økinum: ", self.landSizes,
              "\nRúm: ", self.rooms,"\nHæddir: ", self.floors, "\n")
    def writeToCSV(self):
        print("test")

In [298]:
betriURL = 'https://www.betriheim.fo/'
def scrape_page(driver):
    url = f"{betriURL}"
    driver.get(url)
    time.sleep(2)  # Give the page time to load

    return driver.page_source

# Starts up chrome
driver = webdriver.Chrome()  
try:  
    betriResponse = scrape_page(driver)
finally:
    driver.quit()    


# betriResponse = requests.get(betriURL)
betriSoup = BeautifulSoup(betriResponse, 'html.parser')

skynURL = 'https://www.skyn.fo/ognir-til-soelu'
skynResponse = requests.get(skynURL)
skynSoup = BeautifulSoup(skynResponse.text, 'html.parser')

properties = []

In [299]:
BetriPropertyWrappers = betriSoup.find_all('article', class_="c-property c-card grid")

In [303]:
def CheckValueText(property, attribute, classVal):
    value = property.find(attribute, class_=classVal)
    if value is not None:
        value = value.text
    else:
        value = None
    return value

In [349]:
def addressParse(addresses):
    pattern1 = r"((\w+ )(\w+$))"
    pattern2 = r"(\w+[ ]?[^0-9]+[ ]?[^0-9]+)"
    pattern3 = r"([0-9]+\w?)"
    match1 = re.search(pattern1, addresses)
    match2 = re.search(pattern2, addresses)
    match3 = re.search(pattern3, addresses)
    city = match1.group(3)
    postNum = match1.group(2)
    address = match2.group(1)
    houseNum = match3.group(1)
    yield address, houseNum, city, postNum
    

In [391]:
def priceParse(prices):
    if prices:
        pattern = r"(Kr. )"
        match = re.sub(pattern, "", prices)
        pattern = r"(\.)"
        match = re.sub(pattern, "", match)
        return int(match)

In [392]:
def latestPriceOfferParse(latestPriceOffer):
    if latestPriceOffer:
        pattern = r"(Seinasta boð: Kr. )"
        match = re.sub(pattern, "", latestPriceOffer)
        pattern = r"(\.)"
        match = re.sub(pattern, "", match)
        return int(match)

In [396]:
for property in BetriPropertyWrappers:
    websites = "Betri"
    addresses = str(CheckValueText(property, 'address', "medium"))
    addressEntries = addressParse(addresses)
    # for address, houseNum, city, postNum in addressEntries:
    #     print(address,"----",houseNum,"----",postNum,"----",city)
    prices = CheckValueText(property, 'div', "price")
    priceEntries = int(priceParse(prices))
    # print(priceEntries)
    latestPriceOffer = CheckValueText(property, 'div', "latest-offer")
    latestPriceOfferEntries = latestPriceOfferParse(latestPriceOffer)
    priceOfferValidDate = CheckValueText(property, 'div', "valid")
    print(priceOfferValidDate)
   


None
Galdandi til 03-10-2023, kl 14:00
None
None
None
Galdandi til 11-09-2023, kl 11:00
None
None
None
None
None
None
None
None
None
None
None


In [344]:
for property in BetriPropertyWrappers:
    websites = "Betri"
    addresses = CheckValueText(property, 'address', "medium")
    prices = CheckValueText(property, 'div', "price")
    latestPriceOffer = CheckValueText(property, 'div', "latest-offer")
    priceOfferValidDate = CheckValueText(property, 'div', "valid")
    yearBuilt = CheckValueText(property, 'div', "date")
    insideM2 = CheckValueText(property, 'div',"building-size")
    outsideM2 = CheckValueText(property, 'div', "land-size")
    rooms = CheckValueText(property, 'div', "rooms")
    floors = CheckValueText(property, 'div', "floors")
    prop = FaroesProperties(websites, addresses,prices,LatestPrices,
                            validDates,dates,buildingSizes,
                            landSizes,rooms,floors)
    properties.append(prop)

In [284]:
# for prop in properties:
    # prop.display()